In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
PATH=r"/mnt/data/project_10ab_solo/project/data/processed/sample_processed_10b.csv"

In [ ]:
d=pd.read_csv(PATH,parse_dates=["date"]).sort_values("date").reset_index(drop=True)
d["lag1"]=d["ret"].shift(1)
d["lag5"]=d["ret"].shift(5)
d["roll_mean5"]=d["ret"].shift(1).rolling(5).mean()
d["roll_std5"]=d["ret"].shift(1).rolling(5).std()
d["mom5"]=d["price"].pct_change(5).shift(1)
d["z20"]=(d["ret"].shift(1)-d["ret"].shift(1).rolling(20).mean())/d["ret"].shift(1).rolling(20).std()
d=d.dropna()
X=d[["lag1","lag5","roll_mean5","roll_std5","mom5","z20"]].values
yr=d["ret"].shift(-1).dropna().values
Xr=X[:len(yr)]
yc=(d["ret"].shift(-1)>0).astype(int).dropna().values
Xc=X[:len(yc)]

In [ ]:
n=len(Xr)
split=int(n*0.75)
Xtr,Xte=Xr[:split],Xr[split:]
ytr,yte=yr[:split],yr[split:]
pipe=Pipeline([("s",StandardScaler()),("m",Ridge(alpha=1.0))])
pipe.fit(Xtr,ytr)
p=pipe.predict(Xte)
mae=mean_absolute_error(yte,p)
rmse=(mean_squared_error(yte,p, squared=True)**0.5)
print(mae,rmse)
t=np.arange(len(yte))
plt.figure(); plt.plot(t,yte); plt.plot(t,p); plt.title("pred_vs_truth"); plt.show()

In [ ]:
n=len(Xc)
split=int(n*0.75)
Xtr,Xte=Xc[:split],Xc[split:]
ytr,yte=yc[:split],yc[split:]
pipe=Pipeline([("s",StandardScaler()),("c",LogisticRegression(max_iter=200))])
pipe.fit(Xtr,ytr)
p=pipe.predict(Xte)
acc=accuracy_score(yte,p)
prec=precision_score(yte,p,zero_division=0)
rec=recall_score(yte,p,zero_division=0)
f1=f1_score(yte,p,zero_division=0)
print(acc,prec,rec,f1)
cm=confusion_matrix(yte,p)
plt.figure(); plt.imshow(cm); plt.title("confusion_matrix"); plt.show()